In [35]:
import struct
from dataclasses import dataclass

@dataclass
class RTRadioMessage:
    unit_id: int
    vehicle_rssi: int
    latitude: float
    longitude: float
    speed: float
    heading: float
    gpsc_stat: int
    satellite_gps: int
    vehicle_id: int
    vehicle_city_id: int
    left_turn_signal: bool
    right_turn_signal: bool
    op_status: str
    vehicle_mode: str
    vehicle_class: int
    conditional_priority: int
    veh_diag_value: int

def parse_rt_radio_message(raw_bytes):
    format_string = (
        "<"
        "i"  # unit_id                UL24 + BYTE ???
        "H"  # vehicle_rssi           ??
        "2x"
        "i"  # gps_lat_dddmmmmmm      SLONG
        "i"  # gps_lon_dddmmmmmm      SLONG
        "c"  # velocity_mpsd5         UBYTE
        "c"  # hdg_deg2               UBYTE
        "H"  # gpsc_stat              UWORD
        "I"  # satellite_gps          ULONG
        "H"  # vehicle_id             UWORD?
        "B"  # vehicle_city_id        UBYTE?
        "B"  # veh_mode_op_turn       UBYTE
        "B"  # vehicle_class          UBYTE
        "B"  # conditional_priority   UBYTE
        "2x"
        "i"  # veh_diag_value         UBYTE?
    )
    (
        unit_id,
        vehicle_rssi,
        gps_lat_dddmmmmmm,
        gps_lon_dddmmmmmm,
        velocity_mpsd5,
        hdg_deg2,
        gpsc_stat,
        satellite_gps,
        vehicle_id,
        vehicle_city_id,
        veh_mode_op_turn,
        vehicle_class,
        conditional_priority,
        veh_diag_value,
    ) = struct.unpack(format_string, raw_bytes)
    # convert back to decimal degrees
    latitude = (-1 if gps_lat_dddmmmmmm < 0 else 1) * (
        abs(gps_lat_dddmmmmmm) // 1000000 + abs(gps_lat_dddmmmmmm) % 1000000 / 600000
    )
    longitude = (-1 if gps_lon_dddmmmmmm < 0 else 1) * (
        abs(gps_lon_dddmmmmmm) // 1000000 + abs(gps_lon_dddmmmmmm) % 1000000 / 600000
    )
    # convert back to m/s
    speed = int.from_bytes(velocity_mpsd5, byteorder="little") / 5
    # convert back to deg
    heading = 2 * int.from_bytes(hdg_deg2, byteorder="little") - 1
    # split mode_op_turn
    left_turn_signal = bool(veh_mode_op_turn & 0x01)  # first bit
    right_turn_signal = bool(veh_mode_op_turn & 0x02)  # second bit
    op_status = bin(veh_mode_op_turn & 0x1C)  # 3-5th bit
    vehicle_mode = bin(veh_mode_op_turn & 0xE0)  # 6-8th bit
    return RTRadioMessage(
        unit_id,
        vehicle_rssi,
        latitude,
        longitude,
        speed,
        heading,
        gpsc_stat,
        satellite_gps,
        vehicle_id,
        vehicle_city_id,
        left_turn_signal,
        right_turn_signal,
        op_status,
        vehicle_mode,
        vehicle_class,
        conditional_priority,
        veh_diag_value,
    )

In [39]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv(
    "RTRadioMessages.txt",
    # "~/Downloads/RTRADIO_Messages.txt",
    sep=";",
    names=["timestamp", "topic", "data"],
    # parse timestamp as datetime and use as index
    index_col=["timestamp"],
    parse_dates=["timestamp"],
    # read b'...' as bytes
    converters={
        "data": literal_eval
    },
)

In [40]:
# decode data into RTRadioMessage and add fields as columns
df = pd.concat(
    [
        df,
        pd.DataFrame([parse_rt_radio_message(d) for d in df.data], index=df.index)
    ],
    axis=1
)
df.loc[df.heading < 0, "heading"] = 0
df = df.between_time("13:47:40", "13:59:00")

In [45]:
df.head()

,topic,data,unit_id,vehicle_rssi,latitude,longitude,speed,heading,gpsc_stat,satellite_gps,vehicle_id,vehicle_city_id,left_turn_signal,right_turn_signal,op_status,vehicle_mode,vehicle_class,conditional_priority,veh_diag_value
timestamp,,,,,,,,,,,,,,,,,,,
2022-07-15 13:46:51,GTT/GTT/VEH/EVP/2101/2101FP0733/RTRADIO,b'W\x07\x83\x00\x00\x00\x00\x00\xea\x17\xa8\x0...,8587095,0,44.951003,-92.946008,0.0,0,33738,319429938,1,9,False,True,0b100,0b100000,14,0,0
2022-07-15 13:47:02,GTT/GTT/VEH/EVP/2101/2101FP0733/RTRADIO,b'W\x07\x83\x00\x00\x00\x00\x00\xea\x17\xa8\x0...,8587095,0,44.951003,-92.946008,0.0,0,33738,319429938,1,9,False,True,0b100,0b100000,14,0,0
2022-07-15 13:47:03,GTT/GTT/VEH/EVP/2101/2101FP0733/RTRADIO,b'W\x07\x83\x00\x00\x00\x00\x00\xea\x17\xa8\x0...,8587095,0,44.951003,-92.946008,0.0,0,33738,319429938,1,9,False,True,0b100,0b100000,14,0,0
2022-07-15 13:47:04,GTT/GTT/VEH/EVP/2101/2101FP0733/RTRADIO,b'W\x07\x83\x00\x00\x00\x00\x00\xea\x17\xa8\x0...,8587095,0,44.951003,-92.946008,0.0,0,33738,319429938,1,9,False,False,0b100,0b100000,14,0,0
2022-07-15 13:47:05,GTT/GTT/VEH/EVP/2101/2101FP0733/RTRADIO,b'W\x07\x83\x00\x00\x00\x00\x00\xea\x17\xa8\x0...,8587095,0,44.951003,-92.946008,0.0,0,33738,319429938,1,9,False,False,0b100,0b100000,14,0,0


In [5]:
# offset times based on current time
start_time = df.index.min()
time_offset = pd.Timestamp.now() - start_time
df.index += time_offset

In [ ]:
def get_current_gtfs_realtime(df, vehicle_id=1):
    current_time = pd.Timestamp.now()
    current_data = df.asof(current_time)
    current_gtfs_realtime = {
        "header": {
            "gtfsRealtimeVersion": "1.0",
            "incrementality": "FULL_DATASET",
            "timestamp": int(current_time.timestamp()),
        },
        "entity": [
            {
                "id": f"vehicle_{vehicle_id:04}",
                "vehicle": {
                    "vehicle": {
                        "id": f"{vehicle_id:04}",
                        "label": str(vehicle_id),
                    },
                    "position": {
                        "latitude": current_data.latitude,
                        "longitude": current_data.longitude,
                        "bearing": current_data.heading,
                        "speed": current_data.speed,
                    },
                    "timestamp": int(current_data.name.timestamp()),
                },
            },
        ]
    }
    return current_gtfs_realtime

In [26]:
import requests
import time

polling_rate = 5.0
should_repeat = False

mock_gtfs_realtime_api_url = "http://10.30.2.54:8080/vehiclepositions"
# mock_gtfs_realtime_api_url = "http://3.88.32.226:8080/vehiclepositions"

# device_id: ccf643f8-ecd4-11ec-bb39-aeac87a9eda8"
vehicle_id = 1

while (current_time := pd.Timestamp.now()) <= df.index.max() or should_repeat:
    # reset indexes to current time
    if current_time > df.index.max() and should_repeat:
        start_time = df.index.min()
        time_offset = pd.Timestamp.now() - start_time
        df.index += time_offset
    current_gtfs_realtime = get_current_gtfs_realtime(df, vehicle_id)
    original_time = (
        pd.Timestamp.fromtimestamp(current_gtfs_realtime['header']['timestamp'])
        - time_offset
    ).time()
    print(f"{original_time=}")
    print(current_gtfs_realtime["entity"][0])
    requests.post(mock_gtfs_realtime_api_url, json=current_gtfs_realtime)
    # sleep polling_rate seconds
    time.sleep(polling_rate)

In [28]:
mac_address = "00:00:00:d0:00:01".replace(":", "")[-6:]
unit_id_binary = bin(int(mac_address, 16))[2:].zfill(24)
# unit_id = 0x800000 | mac_address
orrer = f"{8388608:024b}"
orred_unit_id = ""
for i in range(0, 24):
    orred_unit_id += str(int(orrer[i]) | int(unit_id_binary[i]))
unit_id = int(orred_unit_id, 2)

unit_id

13631489

In [ ]:
self.cache.hset(self.redis_key, mapping=new_metadata.dict())

{'description': 'tlltestagency gtfs-realtime device 1',
'category': 'device',
'templateId': 'integrationcom',
'state': 'active',
'deviceId': 'tlltestagency-gtfs-realtime-1',
'attributes': {'serial': '1',
'gttSerial': '1',
'addressMAC': '00:00:00:d0:00:01',
'uniqueId': 'cd7a001c-0001-0000-0000-000000000000',
'preemptionLicense': 'pending',
'integration': 'gtfs-realtime'},
'groups': {'ownedby': ['/tlltestagency/tlltestagency']},
'devices': {'installedat': []}}

metadata = {
    "device_id": "tlltestagency-gtfs-realtime-1",
    "device_address_mac_id": 13631489,
    "device_serial": "1",
    "device_gtt_serial": "1",
    "vehicle_VID": 1,
    "vehicle_class": "10",
    "vehicle_mode": 0,
    "agency_unique_id": "F377A268-88E5-11EC-993D-0AFF99E54CA8",
    "agency_cms_id": "F377A268-88E5-11EC-993D-0AFF99E54CA8",
    "agency_code": "asdf",
    "gpio": "asdf",
    "gpio_timestamp": "asdf",
}

In [32]:
def convert_heading(heading):
    new_heading = int(heading / 2 + 0.5)
    return new_heading.to_bytes(1, byteorder="little", signed=False)

convert_heading(0)

b'\x00'